In [33]:
%load_ext autoreload
%autoreload


In [2]:

import pandas
import numpy


In [11]:

cases = pandas.DataFrame([

    (2, 1, 16, 16000, 0.020),
    (1.1, 1, 16, 16000, 0.020),

    (1.1, 1, 16, 16000, 0.016),
    (1.1, 1, 16, 8000, 0.016),
    
], columns=['buffers', 'channels', 'bits', 'samplerate', 'duration'])

c = cases
c['samples'] = (c.samplerate*c.duration).astype(int)
c['size'] = numpy.ceil(c.buffers*c.channels*numpy.ceil(c.bits/8)*c.samples).astype(int)
c['percent'] = ((c['size'] / 4096) * 100).round(1).astype(str) + '%'

cases = c.set_index(['buffers', 'channels', 'bits', 'samplerate', 'duration'])
cases

samples  size percent
buffers channels bits samplerate duration                       
2.0     1        16   16000      0.020         320  1280   31.2%
1.1     1        16   16000      0.020         320   704   17.2%
                                 0.016         256   564   13.8%
                      8000       0.016         128   282    6.9%

In [12]:
# FFT RAM needs

def fft_ram(samples):
    return 2*samples*2

cases.samples.apply(fft_ram)



buffers  channels  bits  samplerate  duration
2.0      1         16    16000       0.020       1280
1.1      1         16    16000       0.020       1280
                                     0.016       1024
                         8000        0.016        512
Name: samples, dtype: int64

In [18]:

def melspec_size(duration, bands=13):
    per_s = numpy.ceil(1.0/duration)
    size = (per_s * bands * 1)
    return size

cases.reset_index().duration.apply(melspec_size)


0    650.0
1    650.0
2    819.0
3    819.0
Name: duration, dtype: float64

In [19]:
564+1024+819

2407

In [20]:
282+512+819

1613

In [22]:
# Might need to use 8k sampelrate instead of 16k
# 256 FFT probably the way to go?
# If keeping 1 second context window, need MFCC or similar feature reduction. Must store as 8 bit, non-linear comp needed?
# Streaming processing, that only needs 1 frame instead of context window, would be beneficial


In [4]:
# FLASH usage
# fft twiddle, Hann window, mel filters (sparse)
(256*2*2) + (256*2) + (40*40*2)

4736